## Import and Cleanup Data

In [1]:
import pandas as pd
import pickle as pkl
import numpy as np
from sklearn.decomposition import PCA, FactorAnalysis
from factor_analyzer import FactorAnalyzer
from sklearn.cross_decomposition import CCA
import matplotlib.pyplot as plt
from mca import MCA


In [2]:
## If data already preprocessed, load it in, else do preprocessing

try:
    data = pd.read_csv('./data/finaltv.csv', index_col=0)
    test = pd.read_csv('./data/finaltest.csv', index_col=0)
    print("Data successfully loaded")
except Exception:
    label, rev_index, features = pkl.load(open('./data/feature_index_all.pkl','rb'))
    Lindel_training = pd.read_csv("./data/Lindel_training_65bp.csv", sep=',', index_col=0)
    Lindel_test = pd.read_csv("./data/Lindel_test_65bp.csv", sep=',', index_col=0)

    print("Number of labels : ", len(label.keys()))
    print("Number of rev_index : ", len(rev_index.keys()))
    print("Number of features : ", len(features.keys()))

    # column descriptions
    # Lindel_training.iloc[0] # guide sequences
    # Lindel_training.iloc[1:3034] # 3033 binary features [2649 MH binary features + 384 one hot encoded features]
    # Lindel_training.iloc[3034:] # 557 observed outcome frequencies

    # # Merge training and test set for dimensionality reduction
    all_data = pd.concat([Lindel_training, Lindel_test])
    # data_features = all_data.iloc[:, 1:3034]

    # # Clean up data
    features = dict(sorted(features.items(), key=lambda item: item[1]))
    feature_labels = list(features.keys())

    labels = dict(sorted(label.items(), key=lambda item: item[1]))
    class_labels = list(labels.keys())

    one_hot_labels = []
    for i in range(80):
        one_hot_labels.append("nt {}".format(str(int(i / 4) + 1)))

    for i in range(304):
        one_hot_labels.append("2nt {}".format(str(int(i / 16) + 1)))

    one_hot_labels = np.array(one_hot_labels)

    column_labels = np.concatenate((np.array(['Guide Sequence', '65bp']), feature_labels, one_hot_labels, class_labels))

    # Rename columns of test and training set
    Lindel_training = Lindel_training.set_axis(column_labels, axis=1, inplace=False)
    Lindel_test = Lindel_test.set_axis(column_labels, axis=1, inplace=False)

    Lindel_training.to_csv('./data/finaltv.csv')
    Lindel_test.to_csv('./data/finaltest.csv')

    data = pd.concat([Lindel_training, Lindel_test], axis=0)


Data successfully loaded


In [ ]:
data

# RUN DIMENSIONALITY REDUCTIONS

In [3]:
# Do data preprocessing

import pickle as pkl
import os,sys,csv,re
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
from tqdm import tqdm_notebook as tqdm
import pylab as pl
import numpy as np
from datetime import datetime

from keras.models import load_model
from keras.callbacks import EarlyStopping
from keras.layers import Dense, Input, Flatten
from keras.models import Sequential, load_model
from keras.regularizers import l2, l1
from sklearn.model_selection import KFold
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras.callbacks import *

# Make the data preprocessing a deterministic process
np.random.seed(42)

# Define useful functions
def mse(x, y):
    return ((x-y)**2).mean()

def corr(x, y):
    return np.corrcoef(x, y)[0, 1] ** 2

def onehotencoder(seq):
    nt= ['A','T','C','G']
    head = []
    l = len(seq)
    for k in range(l):
        for i in range(4):
            head.append(nt[i]+str(k))

    for k in range(l-1):
        for i in range(4):
            for j in range(4):
                head.append(nt[i]+nt[j]+str(k))
    head_idx = {}
    for idx,key in enumerate(head):
        head_idx[key] = idx
    encode = np.zeros(len(head_idx))
    for j in range(l):
        encode[head_idx[seq[j]+str(j)]] =1.
    for k in range(l-1):
        encode[head_idx[seq[k:k+2]+str(k)]] =1.
    return encode

def kfoldsplits(X):
    """Split annotations"""
    kf = KFold(n_splits=10, shuffle=False)
    splits = []
    for trainIdx, validIdx in kf.split(X):
        splits.append((trainIdx, validIdx))
        
    print("The first index of the first split is ", splits[0][0][0])

    return splits

# Preprocess data
model_data = data.values[:,2:].astype(np.float32)
test_data = test.values[:, 2:].astype(np.float32)
print(model_data.shape, type(model_data))

# Sum up deletions and insertions to
X = model_data[:, :(2649 + 384)]
y = model_data[:, (2649 + 384):]

X_test = test_data[:, :(2649 + 384)]
y_test = test_data[:, (2649 + 384):]

print("X Shape ", X.shape, " | y shape ", y.shape)

# Randomly shuffle data
idx = np.arange(len(y))
np.random.shuffle(idx)
X, y = X[idx], y[idx]
test_idx = np.arange(len(y_test))
np.random.shuffle(test_idx)
X_test, y_test = X_test[test_idx], y_test[test_idx]

print("Now removing samples with only insertion events")
X_deletion, y_deletion = [], []
X_test_deletion, y_test_deletion = [], []

# Remove samples that only have insertion events:
for i in range(model_data.shape[0]):
    if 1> sum(y[i,:536])> 0 :
        y_deletion.append(y[i,:536]/sum(y[i,:536]))
        X_deletion.append(X[i])
        
X_deletion, y_deletion = np.array(X_deletion), np.array(y_deletion)

for i in range(test_data.shape[0]):
    if 1> sum(y_test[i,:536])> 0 :
        y_test_deletion.append(y_test[i,:536]/sum(y_test[i,:536]))
        X_test_deletion.append(X_test[i])
        
X_test_deletion, y_test_deletion = np.array(X_test_deletion), np.array(y_test_deletion)

print("X_test_deletion Shape ", X_test_deletion.shape, " | y_test_deletion shape ", y_test_deletion.shape)



splits = kfoldsplits(X_deletion)
print("Number of train/val splits: ", len(splits))


(4349, 3590) <class 'numpy.ndarray'>
X Shape  (4349, 3033)  | y shape  (4349, 557)
Now removing samples with only insertion events
X_test_deletion Shape  (427, 3033)  | y_test_deletion shape  (427, 536)
The first index of the first split is  423
Number of train/val splits:  10


In [ ]:
test.columns[2:(2649 + 384 + 2)].values.astype(str)

In [ ]:
# 1 : PCA

for i in range(1):

    # Instantiate PCA
    n_components = 910
    pca = PCA()

    # Determine transformed features
    X_train_pca = pca.fit_transform(X_deletion)

    # Determine explained variance using explained_variance_ration_ attribute
    exp_var_pca = pca.explained_variance_ratio_

    # Cumulative sum of eigenvalues; This will be used to create step plot
    # for visualizing the variance explained by each principal component.
    cum_sum_eigenvalues = np.cumsum(exp_var_pca)
    print("Nr of components : ", n_components)
    print("Total Percentage of variance explained = ", cum_sum_eigenvalues[-1] * 100)
    #
    # Create the visualization plot
    #b
    plt.bar(range(0,len(exp_var_pca)), exp_var_pca, alpha=0.5, align='center', label='Individual explained variance')
    plt.step(range(0,len(cum_sum_eigenvalues)), cum_sum_eigenvalues, where='mid',label='Cumulative explained variance')
    plt.ylabel('Explained variance ratio')
    plt.xlabel('Principal component index')
    plt.legend(loc='best')
    plt.tight_layout()
    plt.show()

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(pd.DataFrame(cum_sum_eigenvalues))

In [4]:
pca_components = [140, 180, 237, 355, 697, 1491]
variances = [80, 85, 90, 95, 99, 100]
# % of variance explained: components, 80, 85, 90, 95, 99, 100(up to 6 decimals)

In [ ]:
for i, components in enumerate(pca_components):
    
    # Instantiate PCA
    pca = PCA(n_components=components)

    # Determine transformed features
    X_pca = pca.fit_transform(X_deletion)
    X_test_pca = pca.transform(X_test_deletion)
    
    np.save("./pca/tv_{}_var{}".format(components, variances[i]), X_pca)
    np.save("./pca/test_{}_var{}".format(components, variances[i]), X_test_pca)



In [6]:
# Make results dir
now = datetime.now()
dt_string = now.strftime("%d-%m-%Y %H:%M:%S")
save_dir = os.path.join('./results', dt_string)
os.makedirs(save_dir)

print("Save dir is ", save_dir)


for i in range(len(pca_components)):
    print("VAE {} {}".format(pca_components[i], variances[i]))

    train_split, val_split = splits[0]
    
    X_vae = np.load("./vae/VanillaVAE/latent_features_trainval_beta1.0_latentdim{}.npy".format(pca_components[i]))

    x_train = X_vae[:len(train_split)]
    x_valid = X_vae[len(train_split):]
    
    y_train = y_deletion[train_split]
    y_valid = y_deletion[val_split]
    
    print(x_train.shape, x_valid.shape)

    print(y_train.shape, y_valid.shape)

    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="{}/logs/vae_beta1_latentdim{}".format(save_dir, pca_components[i]))
    
    checkpoint_name = save_dir + '/vae_beta1_latentdim{}_cp{}'.format(pca_components[i], i)
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath= checkpoint_name + '-{epoch:02d}.h5',
        monitor="val_loss",
        verbose=0,
        save_best_only=True,
        save_weights_only=False,
        mode="min")
    
    csv_logger = CSVLogger('{}/vae_beta1_latendim{}.log'.format(save_dir, pca_components[i]), separator=',', append=False)


    model = Sequential()
    model.add(Dense(536,  activation='softmax', input_shape=(x_train.shape[1],), kernel_regularizer=None))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['mse'])
    history = model.fit(x_train, y_train, epochs=100, validation_data=(x_valid, y_valid),
              callbacks=[
                  tensorboard_callback,
                  model_checkpoint_callback,
                  csv_logger], verbose=1)

Save dir is  ./results/28-03-2022 11:46:30
VAE 140 80
(3807, 140) (423, 140)
(3807, 536) (423, 536)
Epoch 1/100
119/119 [==============================] - 0s 2ms/step - loss: 6.2220 - mse: 3.7992e-04 - val_loss: 6.1592 - val_mse: 3.7796e-04
Epoch 2/100
119/119 [==============================] - 0s 1ms/step - loss: 6.1014 - mse: 3.7897e-04 - val_loss: 6.0427 - val_mse: 3.7699e-04
Epoch 3/100
119/119 [==============================] - 0s 1ms/step - loss: 5.9887 - mse: 3.7799e-04 - val_loss: 5.9342 - val_mse: 3.7599e-04
Epoch 4/100
119/119 [==============================] - 0s 2ms/step - loss: 5.8836 - mse: 3.7696e-04 - val_loss: 5.8332 - val_mse: 3.7495e-04
Epoch 5/100
119/119 [==============================] - 0s 2ms/step - loss: 5.7862 - mse: 3.7591e-04 - val_loss: 5.7397 - val_mse: 3.7388e-04
Epoch 6/100
119/119 [==============================] - 0s 2ms/step - loss: 5.6960 - mse: 3.7483e-04 - val_loss: 5.6536 - val_mse: 3.7280e-04
Epoch 7/100
119/119 [==============================] -

119/119 [==============================] - 0s 1ms/step - loss: 4.8432 - mse: 3.5522e-04 - val_loss: 4.8762 - val_mse: 3.5407e-04
Epoch 59/100
119/119 [==============================] - 0s 1ms/step - loss: 4.8422 - mse: 3.5520e-04 - val_loss: 4.8760 - val_mse: 3.5408e-04
Epoch 60/100
119/119 [==============================] - 0s 1ms/step - loss: 4.8412 - mse: 3.5518e-04 - val_loss: 4.8757 - val_mse: 3.5408e-04
Epoch 61/100
119/119 [==============================] - 0s 2ms/step - loss: 4.8402 - mse: 3.5515e-04 - val_loss: 4.8756 - val_mse: 3.5408e-04
Epoch 62/100
119/119 [==============================] - 0s 2ms/step - loss: 4.8392 - mse: 3.5513e-04 - val_loss: 4.8754 - val_mse: 3.5409e-04
Epoch 63/100
119/119 [==============================] - 0s 2ms/step - loss: 4.8383 - mse: 3.5511e-04 - val_loss: 4.8753 - val_mse: 3.5410e-04
Epoch 64/100
119/119 [==============================] - 0s 1ms/step - loss: 4.8374 - mse: 3.5509e-04 - val_loss: 4.8752 - val_mse: 3.5410e-04
Epoch 65/100
119/11

119/119 [==============================] - 0s 1ms/step - loss: 5.1551 - mse: 3.6500e-04 - val_loss: 5.1457 - val_mse: 3.6309e-04
Epoch 16/100
119/119 [==============================] - 0s 1ms/step - loss: 5.1200 - mse: 3.6404e-04 - val_loss: 5.1134 - val_mse: 3.6215e-04
Epoch 17/100
119/119 [==============================] - 0s 1ms/step - loss: 5.0886 - mse: 3.6313e-04 - val_loss: 5.0845 - val_mse: 3.6126e-04
Epoch 18/100
119/119 [==============================] - 0s 2ms/step - loss: 5.0606 - mse: 3.6228e-04 - val_loss: 5.0589 - val_mse: 3.6042e-04
Epoch 19/100
119/119 [==============================] - 0s 2ms/step - loss: 5.0356 - mse: 3.6147e-04 - val_loss: 5.0363 - val_mse: 3.5963e-04
Epoch 20/100
119/119 [==============================] - 0s 2ms/step - loss: 5.0134 - mse: 3.6072e-04 - val_loss: 5.0162 - val_mse: 3.5890e-04
Epoch 21/100
119/119 [==============================] - 0s 1ms/step - loss: 4.9938 - mse: 3.6003e-04 - val_loss: 4.9984 - val_mse: 3.5822e-04
Epoch 22/100
119/11

Epoch 73/100
119/119 [==============================] - 0s 2ms/step - loss: 4.8009 - mse: 3.5395e-04 - val_loss: 4.8679 - val_mse: 3.5356e-04
Epoch 74/100
119/119 [==============================] - 0s 2ms/step - loss: 4.7998 - mse: 3.5391e-04 - val_loss: 4.8679 - val_mse: 3.5356e-04
Epoch 75/100
119/119 [==============================] - 0s 2ms/step - loss: 4.7988 - mse: 3.5388e-04 - val_loss: 4.8679 - val_mse: 3.5356e-04
Epoch 76/100
119/119 [==============================] - 0s 2ms/step - loss: 4.7977 - mse: 3.5385e-04 - val_loss: 4.8680 - val_mse: 3.5357e-04
Epoch 77/100
119/119 [==============================] - 0s 1ms/step - loss: 4.7966 - mse: 3.5382e-04 - val_loss: 4.8679 - val_mse: 3.5357e-04
Epoch 78/100
119/119 [==============================] - 0s 3ms/step - loss: 4.7956 - mse: 3.5379e-04 - val_loss: 4.8680 - val_mse: 3.5357e-04
Epoch 79/100
119/119 [==============================] - 0s 2ms/step - loss: 4.7945 - mse: 3.5375e-04 - val_loss: 4.8681 - val_mse: 3.5357e-04
Epoch 

119/119 [==============================] - 0s 1ms/step - loss: 4.8718 - mse: 3.5560e-04 - val_loss: 4.9073 - val_mse: 3.5439e-04
Epoch 31/100
119/119 [==============================] - 0s 2ms/step - loss: 4.8652 - mse: 3.5537e-04 - val_loss: 4.9026 - val_mse: 3.5419e-04
Epoch 32/100
119/119 [==============================] - 0s 2ms/step - loss: 4.8592 - mse: 3.5517e-04 - val_loss: 4.8985 - val_mse: 3.5404e-04
Epoch 33/100
119/119 [==============================] - 0s 2ms/step - loss: 4.8538 - mse: 3.5500e-04 - val_loss: 4.8949 - val_mse: 3.5391e-04
Epoch 34/100
119/119 [==============================] - 0s 2ms/step - loss: 4.8488 - mse: 3.5485e-04 - val_loss: 4.8917 - val_mse: 3.5380e-04
Epoch 35/100
119/119 [==============================] - 0s 2ms/step - loss: 4.8442 - mse: 3.5472e-04 - val_loss: 4.8890 - val_mse: 3.5372e-04
Epoch 36/100
119/119 [==============================] - 0s 2ms/step - loss: 4.8399 - mse: 3.5460e-04 - val_loss: 4.8865 - val_mse: 3.5365e-04
Epoch 37/100
119/11

...

119/119 [==============================] - 0s 2ms/step - loss: 4.7311 - mse: 3.5093e-04 - val_loss: 4.8693 - val_mse: 3.5284e-04
Epoch 46/100
119/119 [==============================] - 0s 2ms/step - loss: 4.7269 - mse: 3.5079e-04 - val_loss: 4.8686 - val_mse: 3.5282e-04
Epoch 47/100
119/119 [==============================] - 0s 1ms/step - loss: 4.7228 - mse: 3.5065e-04 - val_loss: 4.8680 - val_mse: 3.5280e-04
Epoch 48/100
119/119 [==============================] - 0s 1ms/step - loss: 4.7188 - mse: 3.5051e-04 - val_loss: 4.8674 - val_mse: 3.5279e-04
Epoch 49/100
119/119 [==============================] - 0s 1ms/step - loss: 4.7148 - mse: 3.5038e-04 - val_loss: 4.8668 - val_mse: 3.5277e-04
Epoch 50/100
119/119 [==============================] - 0s 1ms/step - loss: 4.7109 - mse: 3.5024e-04 - val_loss: 4.8664 - val_mse: 3.5277e-04
Epoch 51/100
119/119 [==============================] - 0s 2ms/step - loss: 4.7072 - mse: 3.5011e-04 - val_loss: 4.8661 - val_mse: 3.5276e-04
Epoch 52/100
119/11

119/119 [==============================] - 0s 2ms/step - loss: 6.0886 - mse: 3.7887e-04 - val_loss: 6.0382 - val_mse: 3.7695e-04
Epoch 3/100
119/119 [==============================] - 0s 3ms/step - loss: 5.9628 - mse: 3.7775e-04 - val_loss: 5.9271 - val_mse: 3.7591e-04
Epoch 4/100
119/119 [==============================] - 0s 3ms/step - loss: 5.8450 - mse: 3.7657e-04 - val_loss: 5.8238 - val_mse: 3.7483e-04
Epoch 5/100
119/119 [==============================] - 0s 1ms/step - loss: 5.7349 - mse: 3.7533e-04 - val_loss: 5.7284 - val_mse: 3.7372e-04
Epoch 6/100
119/119 [==============================] - 0s 3ms/step - loss: 5.6324 - mse: 3.7404e-04 - val_loss: 5.6402 - val_mse: 3.7258e-04
Epoch 7/100
119/119 [==============================] - 0s 2ms/step - loss: 5.5370 - mse: 3.7270e-04 - val_loss: 5.5590 - val_mse: 3.7141e-04
Epoch 8/100
119/119 [==============================] - 0s 1ms/step - loss: 5.4486 - mse: 3.7132e-04 - val_loss: 5.4849 - val_mse: 3.7024e-04
Epoch 9/100
119/119 [====

Epoch 60/100
119/119 [==============================] - 0s 2ms/step - loss: 4.2774 - mse: 3.1872e-04 - val_loss: 4.8801 - val_mse: 3.5233e-04
Epoch 61/100
119/119 [==============================] - 0s 2ms/step - loss: 4.2697 - mse: 3.1775e-04 - val_loss: 4.8819 - val_mse: 3.5240e-04
Epoch 62/100
119/119 [==============================] - 0s 2ms/step - loss: 4.2621 - mse: 3.1678e-04 - val_loss: 4.8835 - val_mse: 3.5244e-04
Epoch 63/100
119/119 [==============================] - 0s 2ms/step - loss: 4.2546 - mse: 3.1583e-04 - val_loss: 4.8851 - val_mse: 3.5248e-04
Epoch 64/100
119/119 [==============================] - 0s 2ms/step - loss: 4.2472 - mse: 3.1486e-04 - val_loss: 4.8870 - val_mse: 3.5252e-04
Epoch 65/100
119/119 [==============================] - 0s 3ms/step - loss: 4.2399 - mse: 3.1389e-04 - val_loss: 4.8888 - val_mse: 3.5257e-04
Epoch 66/100
119/119 [==============================] - 0s 2ms/step - loss: 4.2327 - mse: 3.1294e-04 - val_loss: 4.8907 - val_mse: 3.5262e-04
Epoch 

119/119 [==============================] - 0s 3ms/step - loss: 4.3668 - mse: 3.1852e-04 - val_loss: 5.0588 - val_mse: 3.5829e-04
Epoch 18/100
119/119 [==============================] - 0s 4ms/step - loss: 4.3057 - mse: 3.1221e-04 - val_loss: 5.0359 - val_mse: 3.5731e-04
Epoch 19/100
119/119 [==============================] - 0s 4ms/step - loss: 4.2490 - mse: 3.0587e-04 - val_loss: 5.0161 - val_mse: 3.5639e-04
Epoch 20/100
119/119 [==============================] - 0s 2ms/step - loss: 4.1964 - mse: 2.9951e-04 - val_loss: 4.9992 - val_mse: 3.5558e-04
Epoch 21/100
119/119 [==============================] - 1s 5ms/step - loss: 4.1477 - mse: 2.9327e-04 - val_loss: 4.9850 - val_mse: 3.5487e-04
Epoch 22/100
119/119 [==============================] - 0s 3ms/step - loss: 4.1022 - mse: 2.8710e-04 - val_loss: 4.9733 - val_mse: 3.5427e-04
Epoch 23/100
119/119 [==============================] - 0s 4ms/step - loss: 4.0599 - mse: 2.8106e-04 - val_loss: 4.9634 - val_mse: 3.5377e-04
Epoch 24/100
119/11

Epoch 75/100
119/119 [==============================] - 0s 3ms/step - loss: 3.2374 - mse: 1.3266e-04 - val_loss: 5.2055 - val_mse: 3.8314e-04
Epoch 76/100
119/119 [==============================] - 0s 3ms/step - loss: 3.2304 - mse: 1.3153e-04 - val_loss: 5.2131 - val_mse: 3.8402e-04
Epoch 77/100
119/119 [==============================] - 0s 3ms/step - loss: 3.2236 - mse: 1.3035e-04 - val_loss: 5.2205 - val_mse: 3.8494e-04
Epoch 78/100
119/119 [==============================] - 1s 5ms/step - loss: 3.2169 - mse: 1.2920e-04 - val_loss: 5.2281 - val_mse: 3.8589e-04
Epoch 79/100
119/119 [==============================] - 0s 4ms/step - loss: 3.2103 - mse: 1.2811e-04 - val_loss: 5.2356 - val_mse: 3.8684e-04
Epoch 80/100
119/119 [==============================] - 0s 3ms/step - loss: 3.2039 - mse: 1.2698e-04 - val_loss: 5.2427 - val_mse: 3.8775e-04
Epoch 81/100
119/119 [==============================] - 1s 4ms/step - loss: 3.1977 - mse: 1.2597e-04 - val_loss: 5.2502 - val_mse: 3.8868e-04
Epoch 

In [20]:
forecast = pd.read_csv("/home/bram/lindel423/data/Lindel_ForeCasT_testset_4000+samples.txt", sep='\t').values[:,1:].astype(np.float32)

In [23]:
fX = forecast[:, :(2649 + 384)]
fy = forecast[:, (2649 + 384):]

In [25]:
fy.shape

(4737, 557)

In [17]:
# Evaluate Models
trained_model = load_model('/home/bram/lindel423/results/VAE/vae_beta1_latentdim180_cp1-74.h5')
history = pd.read_csv('/home/bram/lindel423/results/VAE/vae_beta1_latendim180.log')
vae_y = np.load('./vae/VanillaVAE/latent_features_test_beta1.0_latentdim180.npy')


In [18]:
y_hat = trained_model.predict(vae_y)
mse(y_hat, y_test_deletion)

0.00034762413

In [14]:
history.iloc[80]

epoch       80.000000
loss         4.824185
mse          0.000355
val_loss     4.874755
val_mse      0.000354
Name: 80, dtype: float64

In [ ]:
# 2 : MCA
"""
Multiple correspondence analysis (MCA) is an extension of correspondence analysis (CA).
It should be used when you have more than two categorical variables.
The idea is simply to compute the one-hot encoded version of a dataset and apply CA on it.
As an example we're going to use the balloons dataset taken from the UCI datasets website.
"""
mca_obj = MCA(data.iloc[:, 1:])

row_factor_scores = mca_obj.fs_r(percent=0.99)
print("Row factor scores (99% variance explained) shape : ", row_factor_scores.shape)

column_factor_scores = mca_obj.fs_c(percent=0.99)
print("Column factor scores (99% variance explained) shape : ", column_factor_scores.shape)


In [ ]:
# 3 : FA

# Instantiate factor analysis object
from factor_analyzer.factor_analyzer import FactorAnalyzer 
fa = FactorAnalyzer(rotation='varimax')
fa.fit(data.iloc[:, 1:])
# Check Eigenvalues
ev, v = fa.get_eigenvalues()
ev